In [7]:
data = open('kafka.txt', 'r').read()
chars = list(set(data))
totalCharacters = len(data)
vocabLen = len(chars)

In [14]:
characterToIndex = {ch:i for i,ch in enumerate(chars)}
print(characterToIndex)
indexToCharacter = {i:ch for i,ch in enumerate(chars)}
print(indexToCharacter)

{'C': 0, 'A': 1, 'J': 2, ' ': 3, 'w': 4, 'k': 5, ';': 6, 'd': 7, '!': 8, 'e': 9, 'u': 10, 'f': 11, 'O': 12, '?': 13, 'ç': 14, 'c': 15, 'W': 16, ')': 17, 'g': 18, 'n': 19, 'y': 20, 'L': 21, '.': 22, 'l': 23, '-': 24, 'r': 25, 'B': 26, 'I': 27, 'q': 28, '\n': 29, 'i': 30, 'v': 31, 'H': 32, 'h': 33, 'o': 34, 'b': 35, 'z': 36, '"': 37, 'm': 38, ',': 39, 't': 40, 'U': 41, 'j': 42, 'G': 43, ':': 44, 'N': 45, 'a': 46, 'D': 47, 'T': 48, 'P': 49, 'Y': 50, '(': 51, 's': 52, 'p': 53, 'Q': 54, 'x': 55, 'E': 56, 'M': 57, 'S': 58, 'V': 59, "'": 60, 'F': 61}
{0: 'C', 1: 'A', 2: 'J', 3: ' ', 4: 'w', 5: 'k', 6: ';', 7: 'd', 8: '!', 9: 'e', 10: 'u', 11: 'f', 12: 'O', 13: '?', 14: 'ç', 15: 'c', 16: 'W', 17: ')', 18: 'g', 19: 'n', 20: 'y', 21: 'L', 22: '.', 23: 'l', 24: '-', 25: 'r', 26: 'B', 27: 'I', 28: 'q', 29: '\n', 30: 'i', 31: 'v', 32: 'H', 33: 'h', 34: 'o', 35: 'b', 36: 'z', 37: '"', 38: 'm', 39: ',', 40: 't', 41: 'U', 42: 'j', 43: 'G', 44: ':', 45: 'N', 46: 'a', 47: 'D', 48: 'T', 49: 'P', 50: 'Y',

In [16]:
import numpy as np
#hyperparameters
learningRate = 0.01
hiddenLayer = 100
seqLength = 9
#modelParameters
#connect input layer to hidden layer
W1 = np.random.randn(vocabLen, hiddenLayer)
# connect hidden layer to output layer
W2 = np.random.randn(hiddenLayer, vocabLen)
# connect hidden layer to hidden layer in the next timestamp
Wr = np.random.randn(hiddenLayer, hiddenLayer)
b1 = np.random.randn(hiddenLayer, 1)
b2 = np.random.randn(vocabLen, 1)

In [ ]:
# the loss function would take in the input chars, the output chars and the previous hidden state
# it outputs the hidden state, the gradients for each parameter between layers and the last hidden states
def propagate(inputChars, outputChars, prevH):
    
    for i in range(len(inputChars)):
        
        h[t] = W1.T.dot(xs[i]) + Wr.dot(h[t-1]) + b1 # h has to be of the same dimension as b
        y[t] = h[t]
    